# Notebook 3, despliegue de modelo en Azure

In [ ]:
import json
import os
from dotenv import load_dotenv
load_dotenv()

# Usamos una variable de entorno oculta para cargar el id de la suscripción, se trata de un archivo .env que debe estar en el mismo directorio que este script
my_secret = os.getenv("MY_SECRET")


my_id = my_secret

from azureml.core import Workspace

ws = Workspace.create(name="workspace4",
                      subscription_id = my_id,
                      resource_group = "act-ml-workflow-3",
                      location = "eastus2")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model2.pkl",
                                  model_name=mname,
                                  workspace=ws)



Deploying StorageAccount with name workspacstorage666ae3284.
Deploying KeyVault with name workspackeyvaultbe466228.
Deploying AppInsights with name workspacinsights65fc8e31.
Deployed AppInsights with name workspacinsights65fc8e31. Took 1.97 seconds.
Deployed KeyVault with name workspackeyvaultbe466228. Took 21.08 seconds.
Deploying Workspace with name workspace4.
Deployed StorageAccount with name workspacstorage666ae3284. Took 22.17 seconds.
Deployed Workspace with name workspace4. Took 21.51 seconds.
Registering model model


In [2]:


################ score.py ###################


scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('model')
  model = joblib.load(model_path)



def run(raw_data):
  try: ## Try la predicción.
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)

    columns_model = [" Borrowing dependency",
    " Current Liability to Current Assets",                       
    " Net Value Per Share (C)",                                  
    " Non-industry income and expenditure/revenue",              
    " Net Value Growth Rate",                                    
    " Continuous interest rate (after tax)"]

    data_dummies = data[columns_model]

    result = model.predict(data_dummies).tolist()
   

    return json.dumps(result)
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()



In [ ]:

from azureml.core.environment import Environment
virtual_env = Environment("env-5-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn']) ## Usar los paquetes correctos


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) 

service = Model.deploy(workspace=ws,
                       name='service-2',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\angel\AppData\Local\Temp\ipykernel_14672\3565990180.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [ ]:

#Publicar el servicio y guardar el URI en un archivo json
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri2.json", "w")
file.write(scoreuri)
file.close()



ACI service creation operation finished, operation "Succeeded"


: 